# Research question 2

## Tongues

### How does the language and country of a movie influence its box office revenue?

The goal is to analyse in more detail how the language(s) and the country of origin of a movie influence it's box office revenue.

### Some intial exploration

In [18]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import chain
from collections import Counter

In [14]:
# set global variables
DATA_PATH = '../data/'
MOVIES_FILE = 'movies_combined_inflated.csv'

In [15]:
# load the data
df_movies = pd.read_csv(DATA_PATH + MOVIES_FILE)
df_movies.head()

,wiki_movie_id,freebase_movie_id,movie_name,movie_release_date,movie_box_office_revenue,movie_runtime,movie_languages,movie_countries,movie_genres,release_year,...,combined_revenue,release_month,release_day,primaryTitle,startYear,averageRating,numVotes,tconst,director,inflated_revenue
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"[('/m/02h40lc', 'English Language')]","[('/m/09c7w0', 'United States of America')]","[('/m/01jfsb', 'Thriller'), ('/m/06n90', 'Scie...",2001.0,...,14010832.0,8.0,24.0,Ghosts of Mars,2001.0,4.9,58868.0,tt0228333,John Carpenter,1.899115e+07
1,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"[('/m/02h40lc', 'English Language')]","[('/m/09c7w0', 'United States of America')]","[('/m/04t36', 'Musical'), ('/m/01z4y', 'Comedy...",1938.0,...,4000000.0,8.0,16.0,Alexander's Ragtime Band,1938.0,6.8,2357.0,tt0029852,Henry King,6.812740e+07
2,171005,/m/016ywb,Henry V,1989-11-08,10161099.0,137.0,"[('/m/02h40lc', 'English Language')]","[('/m/07ssc', 'United Kingdom')]","[('/m/04xvh5', 'Costume drama'), ('/m/082gq', ...",1989.0,...,10161099.0,11.0,8.0,Henry V,1989.0,7.5,32088.0,tt0097499,Kenneth Branagh,1.967252e+07
3,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,139.0,"[('/m/02h40lc', 'English Language')]","[('/m/09c7w0', 'United States of America')]","[('/m/0hj3myq', ""Children's/Family""), ('/m/04t...",1964.0,...,102272727.0,8.0,27.0,Mary Poppins,1964.0,7.8,189183.0,tt0058331,Robert Stevenson,7.913870e+08
4,612710,/m/02wjqm,New Rose Hotel,1999-10-01,21521.0,92.0,"[('/m/02h40lc', 'English Language')]","[('/m/09c7w0', 'United States of America')]","[('/m/01jfsb', 'Thriller'), ('/m/06n90', 'Scie...",1999.0,...,21521.0,10.0,1.0,NaN,NaN,0.0,0.0,NaN,Unknown,3.100822e+04


How many countries/ languages does the dataset have? Which ones are the most common one?

<div class="alert alert-success">

  💭 <strong>Idea:</strong>

  - Analyse the number of countries and languages in the dataset
  - Think about how this distribution could influence/ change further analysis
  
</div>

<div class="alert alert-warning">

  🛠️ <strong>Methods:</strong>
  - Basic prints
  - Visualization: bar plot, pie chart, 
</div>

In [ ]:
# todoooo -> not used and old, check again

def extract_countries(countries):
    """
    Extract the countries from a column containing a list of countries 
    Args:
        countries: list of countries
    Returns:
        list of countries
    """
    if not countries:
        return None
    return ', '.join([country[1] for country in countries if country])


def extract_languages(languages):
    """
    Extract the languages from a column containing a list of languages 
    Args:
        languages: list of languages
    Returns:
        list of languages
    """
    if not languages:
        return None
    return ', '.join([language[1] for language in languages if language])


In [28]:
# extract the countries and languages
df_movies['extracted_countries'] = df_movies['movie_countries'].apply(extract_countries)
df_movies['extracted_languages'] = df_movies['movie_languages'].apply(extract_languages)

# show the number of countries and languages
print('Number of countries:', df_movies['extracted_countries'].nunique())
print('Number of languages:', df_movies['extracted_languages'].nunique())

# top 10 countries and languages
print('Top 10 countries:', df_movies['extracted_countries'].value_counts().head(10))
print('Top 10 languages:', df_movies['extracted_languages'].value_counts().head(10))

IndexError: string index out of range

<div class="alert alert-info">
  💡 <strong>Discussion:</strong>

  - Write here what has been found
</div>

### Language influence

How does the language influence the box office revenue? Do movies in widely spoken languages (for example English, Spanish) tend to have higher revenue?

<div class="alert alert-success">

  💭 <strong>Idea:</strong>

  - Analyse the importance of the language for the box office revenue
  - Maybe only take the primary language
  
</div>

<div class="alert alert-warning">

  🛠️ <strong>Methods:</strong>
  - Statistical hypothesis testing
  - Regression analysis
  
</div>

### Country influence

<div class="alert alert-success">

  💭 <strong>Idea:</strong>

  - Analyse the importance of the country for the box office revenue
  
</div>

<div class="alert alert-warning">

  🛠️ <strong>Methods:</strong>
  - Statistical hypothesis testing
  
</div>

### Relation between language and country

How are language and country correlated? What does this imply for previous and/ or further analysis?

<div class="alert alert-success">

  💭 <strong>Idea:</strong>

  - Analyse the correlation between the country and the language
  - Use the information to guide further anaylsis and to interpret previous one

</div>

<div class="alert alert-warning">

  🛠️ <strong>Methods:</strong>
  - Statistical hypothesis testing
  - Regression analyis
  
</div>

### Multilingual movies

Do multilingual movies have more success (in terms of box office revenues)? Is there a relationship between the number of languages spoken in a movie and its box office revenue?

<div class="alert alert-success">

  💭 <strong>Idea:</strong>

  - Analyse the influence of multilinguality in the movies
  - Keep in mind that if a movie has more than one language listed, it means the languages used in the original version (not the translations later on)

</div>

<div class="alert alert-warning">

  🛠️ <strong>Methods:</strong>
  - XXXX
  
</div>

### Language and genre

How does the language of a movie interact with its genre in terms of box office revenue? Are certain genres more successful in specific languages?